> ### EEE4423: Deep Learning Lab

# Final Project: Long-tail Visual Recognition for Image Classification

<h4><div style="text-align: right"> Due date: June 24, 2022.  </div> <br>
<div style="text-align: right"> Please upload your file @ LearnUs and submit via e-mail by 2 PM in the form of [ID_Name_project.ipynb]. </div></h4>

In [15]:
import os
import os.path as osp
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from misc.project.utils import resnet18, IMBALANCECIFAR10, IMBALANCECIFAR100, compute_accuracy

import warnings
warnings.filterwarnings("ignore", category=UserWarning) ## 경고 무시

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [16]:
print('STEP 6: INSTANTIATE OPTIMIZER CLASS')

DATASET = 'CIFAR10' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'exp' #['exp', 'step'] # 질문
IMB_FACTOR = 0.1 #[0.1, 0.01]
SAVE_DIR = 'logs/baseline/CIFAR10/exp-0.1' 

LR = 0.1
BATCH_SIZE = 128 
MOMENTUM = 0.9
WEIGHT_DECAY = 2e-4
EPOCHS = 100

STEP 6: INSTANTIATE OPTIMIZER CLASS


In [17]:
os.makedirs(osp.join(SAVE_DIR), exist_ok=True)

In [18]:
print('STEP 1: LOADING DATASET')

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

if DATASET == 'CIFAR10':
    train_dataset_0 = IMBALANCECIFAR10(root='../dataset/project', imb_type=IMB_TYPE, imb_factor=IMB_FACTOR, train=True, download=True, transform=transform_train)
    test_dataset = torchvision.datasets.CIFAR10(root='../dataset/project', train=False, download=True, transform=transform_test)
elif DATASET == 'CIFAR100':
    train_dataset_0 = IMBALANCECIFAR100(root='../dataset/project', imb_type=IMB_TYPE, imb_factor=IMB_FACTOR, train=True, download=True, transform=transform_train)
    test_dataset = torchvision.datasets.CIFAR100(root='../dataset/project', train=False, download=True, transform=transform_test)

cls_num_list = train_dataset_0.get_cls_num_list() # 각 class에 몇개가 들어있는지 나타냄 
print('cls num list:')
print(cls_num_list)
num_classes = len(cls_num_list)
print(num_classes)

STEP 1: LOADING DATASET
Files already downloaded and verified
Files already downloaded and verified
cls num list:
[5000, 3871, 2997, 2320, 1796, 1391, 1077, 834, 645, 500]
10


In [19]:
from torch.utils.data import Dataset, DataLoader, random_split
torch.manual_seed(torch.initial_seed())
generator = torch.Generator()
generator.manual_seed(0)
dataset_size = len(train_dataset_0)
validation_size = int(dataset_size * 0.1) if not int(dataset_size * 0.1)==0 else 1 
train_size = dataset_size - validation_size
test_size = 0

train_dataset, validation_dataset, test_dataset_0 = random_split(train_dataset_0, [train_size, validation_size, test_size],generator=generator)

print(len(train_dataset_0))
print(len(train_dataset))
print(len(validation_dataset))
print(len(test_dataset_0))

20431
18388
2043
0


In [20]:
print('STEP 2: MAKING DATASET ITERABLE')
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=4, drop_last=False)

val_loader = torch.utils.data.DataLoader(
    validation_dataset, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=4, drop_last=False)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=100, shuffle=False,
    num_workers=4, )

STEP 2: MAKING DATASET ITERABLE


In [21]:
# 다른 방법이긴하지만 이렇게 표현가능  
list1 = np.zeros(10)
for batch_index, data in enumerate(train_loader):
    image, target = data
    for j in range(len(target)) :
        k =target[j] 
        list1[k]+=1

print(list1)
print(list1.sum())

[4500. 3446. 2684. 2104. 1631. 1254.  974.  757.  582.  456.]
18388.0


In [22]:
print('STEP 3: CREATE MODEL CLASS (VGG16)')

class ResNet18(nn.Module):
    def __init__(self, num_classes):
        super(ResNet18, self).__init__()
        self.backbone = resnet18()
        self.classifier = nn.Linear(512, num_classes)
    
    def forward(self, x):
        batch_size = x.shape[0]
        x = self.backbone(x)
        x = F.adaptive_max_pool2d(x, 1)
        x = x.view(batch_size, -1)
        pred = self.classifier(x)
#         print(pred.size()) # torch.Size([128, 10])
        return pred

STEP 3: CREATE MODEL CLASS (VGG16)


In [23]:
print('STEP 4: INSTANTIATE MODEL CLASS')

model = ResNet18(num_classes)
model = model.cuda()

print('STEP 5: INSTANTIATE LOSS CLASS')
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=150, gamma=0.1)

STEP 4: INSTANTIATE MODEL CLASS
STEP 5: INSTANTIATE LOSS CLASS


In [24]:
torch.cuda.empty_cache()

In [10]:
for epoch in range(EPOCHS):
    loss_history = []
    model.train()
    for batch_index, data in enumerate(train_loader):
        image, target = data
        image, target = image.cuda(), target.cuda()

        pred = model(image)
        loss = criterion(pred, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_history.append(loss.item())

    topk_acc, head_acc, tail_acc = compute_accuracy(val_loader, model)
    loss_mean = np.mean(loss_history)
    scheduler.step()

    print('Epoch: [{:03d}] \t Loss {:.4f} \t Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(epoch+1, loss_mean, topk_acc[0], head_acc[0], tail_acc[0]))
    

Epoch: [001] 	 Loss 4.4817 	 Acc 9.71 	 AccHead 12.11 	 AccTail 1.95
Epoch: [002] 	 Loss 3.7678 	 Acc 14.61 	 AccHead 17.99 	 AccTail 3.68
Epoch: [003] 	 Loss 3.5381 	 Acc 16.71 	 AccHead 20.33 	 AccTail 4.98
Epoch: [004] 	 Loss 3.3571 	 Acc 18.91 	 AccHead 21.67 	 AccTail 9.96
Epoch: [005] 	 Loss 3.1880 	 Acc 22.99 	 AccHead 26.89 	 AccTail 10.39
Epoch: [006] 	 Loss 3.0758 	 Acc 22.23 	 AccHead 25.42 	 AccTail 11.90
Epoch: [007] 	 Loss 2.9630 	 Acc 24.43 	 AccHead 27.69 	 AccTail 13.85
Epoch: [008] 	 Loss 2.8484 	 Acc 25.75 	 AccHead 29.90 	 AccTail 12.34
Epoch: [009] 	 Loss 2.7459 	 Acc 28.67 	 AccHead 32.24 	 AccTail 17.10
Epoch: [010] 	 Loss 2.6549 	 Acc 28.87 	 AccHead 32.91 	 AccTail 15.80
Epoch: [011] 	 Loss 2.5433 	 Acc 29.84 	 AccHead 33.98 	 AccTail 16.45
Epoch: [012] 	 Loss 2.4764 	 Acc 31.37 	 AccHead 35.32 	 AccTail 18.61
Epoch: [013] 	 Loss 2.3736 	 Acc 31.53 	 AccHead 35.65 	 AccTail 18.18
Epoch: [014] 	 Loss 2.2983 	 Acc 32.60 	 AccHead 35.92 	 AccTail 21.86
Epoch: [015

Epoch: [117] 	 Loss 0.3242 	 Acc 39.04 	 AccHead 43.28 	 AccTail 25.32
Epoch: [118] 	 Loss 0.2863 	 Acc 38.48 	 AccHead 41.61 	 AccTail 28.35
Epoch: [119] 	 Loss 0.3084 	 Acc 39.91 	 AccHead 43.81 	 AccTail 27.27
Epoch: [120] 	 Loss 0.2876 	 Acc 37.76 	 AccHead 42.01 	 AccTail 24.03
Epoch: [121] 	 Loss 0.2990 	 Acc 37.86 	 AccHead 41.27 	 AccTail 26.84
Epoch: [122] 	 Loss 0.3101 	 Acc 38.32 	 AccHead 41.81 	 AccTail 27.06
Epoch: [123] 	 Loss 0.3095 	 Acc 38.17 	 AccHead 41.14 	 AccTail 28.57
Epoch: [124] 	 Loss 0.2772 	 Acc 41.29 	 AccHead 45.02 	 AccTail 29.22
Epoch: [125] 	 Loss 0.3008 	 Acc 39.35 	 AccHead 44.28 	 AccTail 23.38
Epoch: [126] 	 Loss 0.3133 	 Acc 38.22 	 AccHead 41.74 	 AccTail 26.84
Epoch: [127] 	 Loss 0.3262 	 Acc 40.21 	 AccHead 45.35 	 AccTail 23.59
Epoch: [128] 	 Loss 0.2939 	 Acc 40.16 	 AccHead 44.62 	 AccTail 25.76
Epoch: [129] 	 Loss 0.2993 	 Acc 38.22 	 AccHead 42.07 	 AccTail 25.76
Epoch: [130] 	 Loss 0.3005 	 Acc 37.56 	 AccHead 40.54 	 AccTail 27.92
Epoch:

In [11]:
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'epoch': epoch},
    osp.join(SAVE_DIR, 'ep{:03d}.pth'.format(epoch+1))
)

In [19]:
##  CIFAR 10 : exp : 0.1
SAVE_DIR = 'logs/baseline/CIFAR10/exp-0.1'
model = ResNet18(num_classes)
model.load_state_dict(torch.load(
    osp.join(SAVE_DIR, 'ep100.pth')
)['model'])
model = model.cuda()
topk_acc, head_acc, tail_acc = compute_accuracy(test_loader, model)

print('Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(topk_acc[0], head_acc[0], tail_acc[0]))

Acc 70.78 	 AccHead 76.90 	 AccTail 64.66


In [20]:
##  CIFAR 10 : exp : 0.01 
SAVE_DIR = 'logs/baseline/CIFAR10/exp-0.01' 
model = ResNet18(num_classes)
model.load_state_dict(torch.load(
    osp.join(SAVE_DIR, 'ep100.pth')
)['model'])
model = model.cuda()
topk_acc, head_acc, tail_acc = compute_accuracy(test_loader, model)

print('Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(topk_acc[0], head_acc[0], tail_acc[0]))

Acc 51.47 	 AccHead 71.48 	 AccTail 31.46


In [21]:
## CIFAR 10 : step : 0.1
SAVE_DIR = 'logs/baseline/CIFAR10/step-0.1' 
model = ResNet18(num_classes)
model.load_state_dict(torch.load(
    osp.join(SAVE_DIR, 'ep100.pth')
)['model'])
model = model.cuda()
topk_acc, head_acc, tail_acc = compute_accuracy(test_loader, model)

print('Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(topk_acc[0], head_acc[0], tail_acc[0]))

Acc 69.09 	 AccHead 79.76 	 AccTail 58.42


In [22]:
## CIFAR 10 : step : 0.01
SAVE_DIR = 'logs/baseline/CIFAR10/step-0.01' 
model = ResNet18(num_classes)
model.load_state_dict(torch.load(
    osp.join(SAVE_DIR, 'ep100.pth')
)['model'])
model = model.cuda()
topk_acc, head_acc, tail_acc = compute_accuracy(test_loader, model)

print('Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(topk_acc[0], head_acc[0], tail_acc[0]))

Acc 48.26 	 AccHead 85.94 	 AccTail 10.58


여기서 부터는 CIFAR 100 임

In [29]:
## CIFAR 100 : exp : 0.1 -> 위에 부터 바꾸고 다시 시작해야함.
SAVE_DIR = 'logs/baseline/CIFAR100/exp-0.1'
model = ResNet18(num_classes)
model.load_state_dict(torch.load(
    osp.join(SAVE_DIR, 'ep100.pth')
)['model'])
model = model.cuda()
topk_acc, head_acc, tail_acc = compute_accuracy(test_loader, model)

print('Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(topk_acc[0], head_acc[0], tail_acc[0]))

Acc 33.60 	 AccHead 42.48 	 AccTail 24.72


In [30]:
## 
SAVE_DIR = 'logs/baseline/CIFAR100/exp-0.01'
model = ResNet18(num_classes)
model.load_state_dict(torch.load(
    osp.join(SAVE_DIR, 'ep100.pth')
)['model'])
model = model.cuda()
topk_acc, head_acc, tail_acc = compute_accuracy(test_loader, model)

print('Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(topk_acc[0], head_acc[0], tail_acc[0]))

Acc 24.69 	 AccHead 38.68 	 AccTail 10.70


In [31]:
## 
SAVE_DIR = 'logs/baseline/CIFAR100/step-0.1'
model = ResNet18(num_classes)
model.load_state_dict(torch.load(
    osp.join(SAVE_DIR, 'ep100.pth')
)['model'])
model = model.cuda()
topk_acc, head_acc, tail_acc = compute_accuracy(test_loader, model)

print('Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(topk_acc[0], head_acc[0], tail_acc[0]))

Acc 34.35 	 AccHead 50.80 	 AccTail 17.90


In [32]:
## 
SAVE_DIR = 'logs/baseline/CIFAR100/step-0.01'
model = ResNet18(num_classes)
model.load_state_dict(torch.load(
    osp.join(SAVE_DIR, 'ep100.pth')
)['model'])
model = model.cuda()
topk_acc, head_acc, tail_acc = compute_accuracy(test_loader, model)

print('Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(topk_acc[0], head_acc[0], tail_acc[0]))

Acc 27.10 	 AccHead 53.22 	 AccTail 0.98


이건 CIFAR100에서 경향성을 알기 위해 epoch200까지 돌린 결과이다. 

In [12]:
## 
SAVE_DIR = 'logs/baseline/CIFAR100/exp-0.1'
model = ResNet18(num_classes)
model.load_state_dict(torch.load(
    osp.join(SAVE_DIR, 'ep200.pth')
)['model'])
model = model.cuda()
topk_acc, head_acc, tail_acc = compute_accuracy(test_loader, model)

print('Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(topk_acc[0], head_acc[0], tail_acc[0]))

Acc 39.99 	 AccHead 50.64 	 AccTail 29.34


실행 시간을 위한 코드

In [25]:
import time 

train_start_time = time.time()

for epoch in range(EPOCHS):
    loss_history = []
    model.train()
    for batch_index, data in enumerate(train_loader):
        image, target = data
        image, target = image.cuda(), target.cuda()

        pred = model(image)
        loss = criterion(pred, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_history.append(loss.item())

    topk_acc, head_acc, tail_acc = compute_accuracy(val_loader, model)
    loss_mean = np.mean(loss_history)
    scheduler.step()

    print('Epoch: [{:03d}] \t Loss {:.4f} \t Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(epoch+1, loss_mean, topk_acc[0], head_acc[0], tail_acc[0]))

train_end_time = time.time()

print(train_end_time-train_start_time)

Epoch: [001] 	 Loss 2.5427 	 Acc 38.96 	 AccHead 49.17 	 AccTail 0.00
Epoch: [002] 	 Loss 1.6652 	 Acc 45.96 	 AccHead 57.75 	 AccTail 0.94
Epoch: [003] 	 Loss 1.5391 	 Acc 47.04 	 AccHead 55.34 	 AccTail 15.33
Epoch: [004] 	 Loss 1.4422 	 Acc 53.21 	 AccHead 62.01 	 AccTail 19.58
Epoch: [005] 	 Loss 1.3590 	 Acc 52.96 	 AccHead 61.33 	 AccTail 20.99
Epoch: [006] 	 Loss 1.3086 	 Acc 54.82 	 AccHead 63.62 	 AccTail 21.23
Epoch: [007] 	 Loss 1.2369 	 Acc 57.66 	 AccHead 64.67 	 AccTail 30.90
Epoch: [008] 	 Loss 1.2075 	 Acc 59.47 	 AccHead 65.23 	 AccTail 37.50
Epoch: [009] 	 Loss 1.1447 	 Acc 58.69 	 AccHead 65.23 	 AccTail 33.73
Epoch: [010] 	 Loss 1.1038 	 Acc 61.63 	 AccHead 69.18 	 AccTail 32.78
Epoch: [011] 	 Loss 1.0532 	 Acc 62.95 	 AccHead 68.93 	 AccTail 40.09
Epoch: [012] 	 Loss 1.0076 	 Acc 66.13 	 AccHead 72.58 	 AccTail 41.51
Epoch: [013] 	 Loss 0.9664 	 Acc 63.97 	 AccHead 70.91 	 AccTail 37.50
Epoch: [014] 	 Loss 0.9362 	 Acc 63.97 	 AccHead 67.26 	 AccTail 51.42
Epoch: [

In [26]:
train_start_time = time.time()

topk_acc, head_acc, tail_acc = compute_accuracy(test_loader, model)

print('Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(topk_acc[0], head_acc[0], tail_acc[0]))

train_end_time = time.time()

print(train_end_time-train_start_time)

Acc 69.51 	 AccHead 79.24 	 AccTail 59.78
4.727281332015991
